In [44]:
import pandas as pd
import re

def cleaning(s):
  s = str(s)
  s = re.sub(', ', ',', s)
  s = re.sub(',', ' ', s)
  return s

df = pd.read_csv("./datasets/final.csv", encoding="utf-8")
df = df.dropna()
text_data = open('./datasets/slogans.txt', 'w', encoding="utf-8")
for idx, item in df.iterrows():
  slogans = cleaning(item['company']) + ', ' + item['slogan']+ '\n'
  text_data.write(slogans)
text_data.close()

In [2]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset


def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, 
        mlm=mlm,
    )
    return data_collator

def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir, legacy_format=False)
   
  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )
      
  trainer.train()
  trainer.save_model()

train_file_path = './datasets/slogans.txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = './models2'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 5.0
save_steps = 500

train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

 49%|████▉     | 500/1020 [01:46<01:52,  4.61it/s]

{'loss': 2.4771, 'learning_rate': 2.5490196078431373e-05, 'epoch': 2.45}


 98%|█████████▊| 1000/1020 [03:36<00:04,  4.92it/s]

{'loss': 1.6098, 'learning_rate': 9.80392156862745e-07, 'epoch': 4.9}


100%|██████████| 1020/1020 [03:42<00:00,  4.59it/s]


{'train_runtime': 222.1263, 'train_samples_per_second': 4.592, 'epoch': 5.0}


In [24]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
    return tokenizer


def generate_text(sequence, max_length):
    model_path = "./models2"
    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence},', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.pad_token_id,
        top_k=50,
        top_p=0.95,
    )
    return(tokenizer.decode(final_outputs[0], skip_special_tokens=True))

# sequence = input()
# max_len = int(input())

company = "닥스" #(최대 10글자)
context = "셔츠 제조업" #(최대 글자 지정예정)

input = company + ', ' + context 

sequence = input
max_len = 50

print('input :' + sequence)


for i in range(10):
    print('=' * 50)
    g = generate_text(sequence, max_len)
    g = g.split('<eos>')[0]
    g = g.split(',')[2:]
    g = ', '.join(g)
    print(g)
    
print('=' * 50)
        

input :닥스, 셔츠 제조업
 이뻐지자
 봄을 즐겨요
 바람막이로써 바람막이를
 스포츠 의류업비거리,  상식을 뒤집다
 누구나 지금 웃고 있습니다.
지금까지 누우자,  전시,  컨벤션 및 행사 대행업Love you are
 다운자신감이 느껴진다
 당신에게 선물하세요
 셔츠 제조업요즘에는 셔츠로 더 사나워
 캐주얼한 의상은 당신의 마음을 움직이다
 운동 및 경기용품 소매업스타일은 스타일대로


In [ ]:
 # 슬로건, 설명 분리를 하지않을시 특정 input 에서 문제 발생'
 
 '''
input :어쌔씬크리드 발할라, 액션게임
==================================================
 액션을 넘어 액션까지
==================================================
 액션 어쌔씬하게 살아나다
==================================================
 액션게임의 새로운 장르
==================================================
 이제 액션할 때
==================================================
 그래서 액션이다
==================================================
 액션을 넘나들다
==================================================
 액션에 빠지다
==================================================
 그 어디에서도 볼 수 없었던 리얼로드,  액션의 신세계
==================================================
 액션이제 액션도 액션이니까
==================================================
 액션이것이 액션의 시작
==================================================


 input :벼룩마켓, 구직 신문
==================================================
 구인구직 정보 플랫폼취업,  구직 정보 플랫폼취업,  구직 정보 플랫폼취업,  구직 정보 플랫폼취업,  구직 정보 플랫폼취업의 기준은
==================================================
 방송 프로그램 배너광고가 무슨 의미가 있나요?
==================================================
 온라인 정보매개 사이트올 추석에도 매장은 즐거운 기분
==================================================
 잡지,  인쇄업일취월장,  구인구직 정보 플랫폼
==================================================
 광고 대행업스펙은 스펙대로
==================================================
 데이터베이스 제공업선진국 온오프라인에서 구직 정보를 얻다
==================================================
 구인구직 정보 플랫폼취업이 쉬워진다
==================================================
 사진 정보 제공업모두의 취업 플랫폼
==================================================
 잡지 판매업나를 기다리는 많은 구직 정보
==================================================
 데이터베이스 및 온라인정보 제공업내가 원하는 구직 정보 플랫폼
==================================================


 input :커피팩토리, 정말 맛있는 커피와 원두를 즐길 수 있는 공간
==================================================
 세계특허를 얻다
==================================================
 프리미엄 커피를 위한 프리미엄 커피
==================================================
 로스팅!
==================================================
 콜롬비아나에서 찾은 커피 한잔
==================================================
 이 곳은 카페
==================================================
 이것이 스페샬 오디너의 공식
==================================================
 원두커피의 향
==================================================
 오직 로스팅만으로 충분하다
==================================================
 세계 최저 수준의 카페인
==================================================
 원두토마스
==================================================


input :닥스, 셔츠 제조업
==================================================
 이뻐지자
==================================================
 봄을 즐겨요
==================================================
 바람막이로써 바람막이를
==================================================
 스포츠 의류업비거리,  상식을 뒤집다
==================================================
 누구나 지금 웃고 있습니다.
지금까지 누우자,  전시,  컨벤션 및 행사 대행업Love you are
==================================================
 다운자신감이 느껴진다
==================================================
 당신에게 선물하세요
==================================================
 셔츠 제조업요즘에는 셔츠로 더 사나워
==================================================
 캐주얼한 의상은 당신의 마음을 움직이다
==================================================
 운동 및 경기용품 소매업스타일은 스타일대로
==================================================
'''